In [1]:
import pandas as pd
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import preprocess_input
from collections import OrderedDict

In [2]:
import cv2
import os
import pandas as pd

# Load the CSV file
csv_path = 'radiological_gradings.csv'
radiological_data = pd.read_csv(csv_path)

# Initialize lists for each label
pfirrman_grades = []
modic_types = []
up_endplates = []
low_endplates = []
disc_herniation = []
disc_narrowing = []
disc_bulging = []
spondylolisthesis = []
image_list = []  # For storing image data
image_names = []

# Directory containing the disc images
image_dir = "split_discs"

# Loop through each image in the directory
for filename in os.listdir(image_dir):
    if filename.endswith(".png"):
        # Parse patient number and IVD label from the filename
        base_name = filename.replace(".png", "")
        
        # Split the filename and handle cases with additional parts
        parts = base_name.split("_")
        patient_id = int(parts[0])           # First part is always the patient ID
        scan_type = parts[-2]                # Second last part is the scan type (t1/t2)
        ivd_label = int(parts[-1].replace("disc", ""))  # Last part is the disc label
        
        # Find the corresponding row in the CSV
        row = radiological_data[(radiological_data['Patient'] == patient_id) &
                                (radiological_data['IVD label'] == ivd_label)]
        
        # Ensure the row exists and extract label values
        if not row.empty:
            # Append each label to its respective list
            modic_types.append(row['Modic'].values[0])
            up_endplates.append(row['UP endplate'].values[0])
            low_endplates.append(row['LOW endplate'].values[0])
            spondylolisthesis.append(row['Spondylolisthesis'].values[0])
            disc_herniation.append(row['Disc herniation'].values[0])
            disc_narrowing.append(row['Disc narrowing'].values[0])
            disc_bulging.append(row['Disc bulging'].values[0])
            pfirrman_grades.append(row['Pfirrman grade'].values[0])
            
            # Optionally load the image data if needed
            image = Image.open(os.path.join(image_dir, filename))
            # Resize image to 224x224 pixels
            image = image.resize((224,224))
            # Convert image to RGB
            image = image.convert("RGB")
            # Convert the image to numpy array
            image_array = np.array(image)
            # Normalize array
            image_array = preprocess_input(image_array)
            image_list.append(image_array)
            image_names.append(filename)

print("Labels have been mapped to each image and stored in the corresponding lists.")

Labels have been mapped to each image and stored in the corresponding lists.


In [3]:
image_list_np = np.array(image_list)
pfirrman_grades_np = np.array(pfirrman_grades)
pfirrman_grades_np = np.round(pfirrman_grades_np).astype(int) - 1
modic_types_np = np.array(modic_types)

In [4]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling to reduce dimensions
x = Dense(1024, activation='relu')(x)  # Fully connected layer
pfirrman_predictions = Dense(5, activation='softmax', name='pfirrman_output')(x)
modic_predictions = Dense(4, activation='softmax', name='modic_output')(x)

In [5]:
model = Model(inputs=base_model.input, outputs=[pfirrman_predictions, modic_predictions])
model.compile(optimizer='adam', loss={
        'pfirrman_output': 'sparse_categorical_crossentropy', 
        'modic_output': 'sparse_categorical_crossentropy'
    },          
    metrics={
        'pfirrman_output': 'accuracy',
        'modic_output': 'accuracy'
    }
)

In [6]:
history = model.fit(
    image_list_np,            # Input images
    [pfirrman_grades_np, modic_types_np],
    batch_size=32,
    epochs=10,
    validation_split=0.2,     # Split 20% of data for validation
    verbose=1                 # Set to 1 to see training progress
)

Epoch 1/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - loss: 4.0199 - modic_output_accuracy: 0.5682 - modic_output_loss: 2.0582 - pfirrman_output_accuracy: 0.2832 - pfirrman_output_loss: 1.9607 - val_loss: 2.7074 - val_modic_output_accuracy: 0.5234 - val_modic_output_loss: 1.1882 - val_pfirrman_output_accuracy: 0.3458 - val_pfirrman_output_loss: 1.5054
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - loss: 1.8530 - modic_output_accuracy: 0.7155 - modic_output_loss: 0.5593 - pfirrman_output_accuracy: 0.4796 - pfirrman_output_loss: 1.2933 - val_loss: 2.5895 - val_modic_output_accuracy: 0.5093 - val_modic_output_loss: 0.9337 - val_pfirrman_output_accuracy: 0.2710 - val_pfirrman_output_loss: 1.6325
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - loss: 1.5557 - modic_output_accuracy: 0.7724 - modic_output_loss: 0.4756 - pfirrman_output_accuracy: 0.5606 - pfirrman_output_loss: 1.0801 - val_loss: 2.9819 - val_modic_output_accuracy: 0.5187 - val_modic_output_loss: 1.1948 - val_pfirrman_out

In [14]:
test_image_path = 'split_discs/131_t2_disc_4.png'

# Load, resize, and preprocess the test image
img = Image.open(test_image_path).convert('RGB')
img_resized = img.resize((224, 224))
img_array = np.array(img_resized)

# Preprocess the image for ResNet50
img_preprocessed = tf.keras.applications.resnet50.preprocess_input(img_array)
img_preprocessed = np.expand_dims(img_preprocessed, axis=0)  # Add batch dimension

# Run the prediction
predicted_probabilities = model.predict(img_preprocessed)
predicted_pfirrman = np.argmax(predicted_probabilities[0]) + 1  # Add 1 if classes are 1-5
predicted_modic = np.argmax(predicted_probabilities[1])
print(f"Predicted Pfirrman grade: {predicted_pfirrman}")
print(f"Predicted Modic: {predicted_modic}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Predicted Pfirrman grade: 1
Predicted Modic: 0


In [13]:
idx = 164
print(f"  Image name: {image_names[idx]}")
print(f"  Pfirrman Grade: {pfirrman_grades[idx]}")
print(f"  Modic Type: {modic_types[idx]}")
print(f"  UP Endplate: {up_endplates[idx]}")
print(f"  LOW Endplate: {low_endplates[idx]}")
print(f"  Spondylolisthesis: {spondylolisthesis[idx]}")
print(f"  Disc Herniation: {disc_herniation[idx]}")
print(f"  Disc Narrowing: {disc_narrowing[idx]}")
print(f"  Disc Bulging: {disc_bulging[idx]}")

  Image name: 131_t2_disc_4.png
  Pfirrman Grade: 1
  Modic Type: 0
  UP Endplate: 0
  LOW Endplate: 0
  Spondylolisthesis: 0
  Disc Herniation: 0
  Disc Narrowing: 0
  Disc Bulging: 0


In [15]:
model.save('nonBinaryIndividualPredictions.keras')